# This module to obtain the partial annotation by using the GNN model to match neurons cross frames
1. Produce the segmentation from 3D stardist
2. Obtain the nodes features and edges for each frame
3. Get the order of the frame (parent and child frame order) \
    (1) The fist way is to run the frame_order_zephir.py  \
    (2) if you alread have he order of the tracking as list, igore this step and use it as list. eg:t_fram = [100,103,105,102,104] 
4. Matching the neurons in two pairs of frame
5. Otput the paired neurons coordinates in the annotaiton.h5

## Preparation: install tensorflow, pytroch
Please let me know if any package you need can not be installed

In [1]:
import time
t0 = time.time()

## 1: Extract the features from the segmented files, convert into graph, save it as pt
make sure you have already segmented the volume and load the path of folder

In [2]:
from MatchPartial.Graph_data import *
from parameters import *

In [3]:
print("packages loaded")

packages loaded


In [4]:
path_graph = 'graph/'  ## the saving path of graph for each time index, the graph path contain nodes and edge features for each frame
pathlib.Path(path_graph).mkdir(parents=True, exist_ok=True) 

In [5]:
# # t_max = 2
# def save_data(t_idx):
#     [ch,num_nearest] = [1,5] ## ch is which channel to load the image, for NeuroPal please convert into RGB then use it
#     data = get_data_generator(t_idx, ch, num_nearest,img_h5_path,seg_h5_path,annotation_path,train_val)
#     torch.save(data, f'{path_graph}{t_idx}.pt')

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(save_data, tqdm(range(0,t_max )))

## 2. Get the tracking frame order
Note: Pass this step and directly run the matching function if you have the order of the tracking frames

In [6]:
from frame_order_zephir import *
from parameters import *

Checkpoint updated for ['container'] @ [2024-03-28 00:12:22.207819]
Using frame #444 as initial reference with 178 annotations found...

Annotations loaded for frames [444] with shape: (1, 178, 3)
Checkpoint updated for ['container'] @ [2024-03-28 00:12:22.355568]
Checkpoint updated for ['results'] @ [2024-03-28 00:12:22.408577]

Compiling models and loss function...
******* ERROR: Zephod checkpoint file not found!
Checkpoint updated for ['container'] @ [2024-03-28 00:12:22.477088]
Checkpoint updated for ['zephir_kwargs', 'zephir', 'zephod'] @ [2024-03-28 00:12:22.539936]

Building neighbor tree...
Checkpoint updated for ['container'] @ [2024-03-28 00:12:22.602141]

Building frame correlation graph...

Sorting frames by similarity to parent...

Frames sorted with max/mean distance	0.5786 / 0.0178
Checkpoint updated for ['container'] @ [2024-03-28 00:12:23.392878]


In [7]:
print("the tracking frame order list that zephir is using", t_frame)

the tracking frame order list that zephir is using [444, 443, 489, 442, 488, 445, 446, 486, 441, 490, 491, 492, 407, 408, 409, 406, 440, 485, 530, 529, 528, 527, 526, 482, 483, 484, 439, 438, 531, 532, 533, 534, 535, 536, 537, 591, 589, 590, 588, 586, 587, 585, 584, 583, 582, 581, 525, 524, 481, 437, 592, 593, 594, 595, 596, 546, 545, 547, 548, 549, 551, 552, 553, 556, 557, 558, 550, 555, 554, 511, 512, 510, 513, 514, 515, 516, 517, 518, 509, 520, 464, 462, 463, 508, 503, 506, 505, 504, 507, 456, 457, 502, 454, 501, 458, 604, 603, 602, 600, 601, 605, 611, 613, 616, 615, 614, 606, 607, 608, 609, 610, 498, 617, 618, 620, 619, 621, 622, 624, 623, 625, 627, 626, 628, 634, 632, 638, 637, 639, 640, 635, 631, 630, 629, 636, 641, 633, 642, 612, 499, 500, 567, 568, 569, 570, 565, 564, 563, 562, 561, 559, 599, 598, 974, 976, 977, 978, 975, 980, 979, 981, 982, 983, 984, 985, 986, 987, 973, 972, 971, 962, 965, 964, 961, 960, 963, 988, 990, 989, 992, 991, 993, 995, 998, 999, 1000, 994, 996, 997, 10

## 3. Getting the link and store them into the annotations

In [8]:
from MatchPartial.eval_func import *
from parameters import *

In [9]:
# path_graph = 'graph/' 
# nearby_search_num_list = [5]
# model_all = load_model_args(model_path, nearby_search_num_list,with_AM, device,train_val)
# df_all = model_match_patial_neurons(path_graph,t_frame,model_all,img_shape,threshold,nearby_search_num_list,with_AM,device,train_val)
# save_pandas_h5(save_matched_path, df_all)
# print("Output all the matched annotaiton in the path: ", save_matched_path)

In [10]:
# print("The time of running the whole segmented datasets: ",t1-t0)
# print("the number of found partial annotations: ",len(df_all)/(np.max(t_frame)+1))

## 4. The following is based on the zephir running
4.1 The matched annotation from the step 3 is assuming each neuron is well segmented. In the real cases, some are merged in the z direction, leading the mismatch in the next frame. So we find out the annotations that are in the merged neurons and remove these connectivity. \
4.2 The annotation is in the merged neurons, which means there are multiple annotations obtained from the zephir tracking and used as parent frame annotaiton in the segmented neuron. Once the merged segmented neurons are located, the center of it is the megred annotation that needs to be removed.

## runing test

In [11]:
dataset_path

'/work/venkatachalamlab/Hang/GNN_matching_git/code/GNN_match/ZM9624/'

In [12]:
dataset = Path(dataset_path)
annotation = get_annotation_df(dataset)
idx = 500
annotation[annotation['t_idx']==t_frame[idx]]

,id,parent_id,provenance,t_idx,worldline_id,x,y,z
123532,123533,739,b'ZEIR',694,0,0.5,0.5,0.5
123533,123534,739,b'ZEIR',694,1,0.5,0.5,0.5
123534,123535,739,b'ZEIR',694,2,0.5,0.5,0.5
123535,123536,739,b'ZEIR',694,3,0.5,0.5,0.5
123536,123537,739,b'ZEIR',694,4,0.5,0.5,0.5
...,...,...,...,...,...,...,...,...
123705,123706,739,b'ZEIR',694,174,0.5,0.5,0.5
123706,123707,739,b'ZEIR',694,175,0.5,0.5,0.5
123707,123708,739,b'ZEIR',694,176,0.5,0.5,0.5
123708,123709,739,b'ZEIR',694,177,0.5,0.5,0.5


## runing test

In [13]:
rescale = np.array(img_shape) - 1
path_graph = 'graph/' 
seg_path = '/work/venkatachalamlab/Hang/Matching_neuron/seg/'
nearby_search_num_list = [5]
selected_index = torch.tensor([8,7,2],dtype = torch.long)
model_all = load_model_args(model_path, nearby_search_num_list,with_AM, device,train_val)
annotation_orig = get_annotation_df(dataset)
annotation = copy.deepcopy(annotation_orig)
annotation[['x','y','z']] = 0.5
annotation[annotation['t_idx']==444] = annotation_orig[annotation_orig['t_idx']==444] 

### initialize the tracking results and annotations
shape_t = container.get('shape_t')
shape_n = container.get('shape_n')
results = np.zeros((shape_t, shape_n, 3))
t_ref=eval(args['--t_ref']) if args['--t_ref'] else None,
wlid_ref=eval(args['--wlid_ref']) if args['--wlid_ref'] else None,
n_ref=int(args['--n_ref']) if args['--n_ref'] else None,


# for i in tqdm(range(2)):
for i in range(len(t_frame)-1):
    t1 = t_frame[i] ## parent time index
    t2 = t_frame[i+1] ## child time index

    
    ### get the annotation at time t2 from GNN match
    removed_seg_ID = find_merged_index_graphdata(seg_path,t1,img_shape,annotation)
    df = model_pair_neurons_zephir_frame(path_graph,t1,t2,removed_seg_ID,selected_index,model_all,img_shape,threshold,nearby_search_num_list,with_AM,device,train_val)
    index, df_filtered = matched_df(df,annotation,t1,t2,rescale)
    
    print("originalally found matches:", len(df), "after filtered the merged neurons, found matches: ",len(df_filtered))

    ## only take the unique worldline
    # annotation[annotation['t_idx']==t2].iloc[index][['x','y','z']] = df_filtered[['x','y','z']].values
    # annotation[annotation['t_idx']==t2].iloc[index]['provenance'] = b'GNN'
    # annotation.loc[annotation['t_idx'] == t2, 'parent_id'] = t1
    
    indices = annotation[annotation['t_idx'] == t2].index
    annotation.loc[indices[index], ['x', 'y', 'z']] = df_filtered[['x', 'y', 'z']].values
    annotation.loc[indices[index], ['provenance']] = b'GNN'
    annotation.loc[indices[index], ['parent_id']] = t1
    print("the updated annotation at time t1",np.sum(abs(annotation[annotation['t_idx'] == t1][['x','y','z']].values-0.5)))
    print("the updated annotation at time t2",np.sum(abs(annotation[annotation['t_idx'] == t2][['x','y','z']].values-0.5)))

    
    
    ### how to update the annotation in the containter?
    t_ref = t_frame[0:i+2]
    print("before loading",np.sum(results[t2]))
    container, results = build_partial_annotations(container,annotation,results,t_ref,None,None,)
    print("after loading",np.sum(results[t2]))
    container, results = track_all(
        container=container,
        results=results,
        zephir=zephir,
        zephod=zephod,
        clip_grad=float(args['--clip_grad']),
        lambda_t=float(args['--lambda_t']),
        lambda_d=float(args['--lambda_d']),
        lambda_n=float(args['--lambda_n']),
        lambda_n_mode='norm',
        lr_ceiling=float(args['--lr_ceiling']),
        # lr_coef=2.0,
        lr_floor=float(args['--lr_floor']),
        motion_predict=bool(args['--motion_predict']),
        n_epoch=int(args['--n_epoch']),
        n_epoch_d=int(args['--n_epoch_d']),
        _t_list= np.array([t2]),
    )
    print("!!time",t2, results[t2][0])
    
    ### save the tracking results into dataset
    
    
    
    save_annotations(
    container=container,
    results=results,
    save_mode='o')
    
    # save_annotations_filename(
    # container=container,
    # results=results,
    # filename='annotations.h5',
    # save_mode='o')
    
    annotation = get_annotation_df(dataset)

originalally found matches: 0 after filtered the merged neurons, found matches:  0
the updated annotation at time t1 0.0
the updated annotation at time t2 0.0
before loading 0.0
Using frame #443 as initial reference with 178 annotations found...
t_annot: [443 444]

Annotations loaded for frames [443, 444] with shape: (2, 178, 3)
Checkpoint updated for ['container'] @ [2024-03-28 00:12:24.584599]
Checkpoint updated for ['results'] @ [2024-03-28 00:12:24.633531]
after loading 0.0


Analyzing movie:   0%|          | 0/1 [00:00<?, ?frames/s]


Frame #443		Parent #444		Reference #443		Distance to parent: d=0.0107



Analyzing movie: 100%|██████████| 1/1 [00:01<00:00,  1.66s/frames]


!!time 443 [ 1.29519685e-05 -3.35603982e-05  5.85720227e-06]

Compiling and saving results to file...
originalally found matches: 0 after filtered the merged neurons, found matches:  0
the updated annotation at time t1 0.004662931
the updated annotation at time t2 0.0
before loading 0.0
Using frame #443 as initial reference with 178 annotations found...
t_annot: [443 444 489]

Annotations loaded for frames [443, 444] with shape: (2, 178, 3)
Checkpoint updated for ['container'] @ [2024-03-28 00:12:27.549368]
Checkpoint updated for ['results'] @ [2024-03-28 00:12:27.633485]
after loading 0.0


...


Analyzing movie: 100%|██████████| 1/1 [00:01<00:00,  1.45s/frames]


!!time 440 [-0.00936973 -0.00266593  0.02746041]

Compiling and saving results to file...
originalally found matches: 0 after filtered the merged neurons, found matches:  0
the updated annotation at time t1 3.5151556
the updated annotation at time t2 0.0
before loading 0.0
Using frame #443 as initial reference with 178 annotations found...
t_annot: [406 407 408 409 440 441 442 443 444 445 446 485 486 488 489 490 491 492]

Annotations loaded for frames [443, 444] with shape: (2, 178, 3)
Checkpoint updated for ['container'] @ [2024-03-28 00:13:07.237455]
Checkpoint updated for ['results'] @ [2024-03-28 00:13:07.282121]
after loading 0.0


Analyzing movie:   0%|          | 0/1 [00:00<?, ?frames/s]


Frame #485		Parent #440		Reference #443		Distance to parent: d=0.0057



Analyzing movie:   0%|          | 0/1 [00:00<?, ?frames/s]

KeyboardInterrupt

